In [ ]:
# 데이터 읽기
import pandas as pd
df_source = pd.read_csv("../Datasets/housing2.csv") # Write your code here.

In [ ]:
# 데이터 보기
df_source.head() 

In [ ]:
df_source.info() 

In [ ]:
# 숫자 데이터 보기 
df_source.describe() 

In [ ]:
# 히스토그램 그리기
import matplotlib.pyplot as plt
df_source.hist(bins=50, figsize=(20,15)) # Write your code here.
plt.show()

In [ ]:
# 카테고리 데이터, 데이터 종류별 갯수보기 
df_source["CHAS"].value_counts() 

In [ ]:
# 두 데이터간 관계도 보기 
from pandas.plotting import scatter_matrix
scatter_matrix(df_source, figsize=(20, 20)); 

In [ ]:
# 데이터 정렬하기 
df_source.corr()["CHAS"].sort_values(ascending=False)

In [ ]:
# df_source.plot(kind="scatter", x ="longitude", y ="latitude", 
#                alpha=0.4, s=df_source["population"]/100, label="population", 
#                figsize=(10,7), c="median_house_value", cmap=plt.get_cmap("jet"), 
#                colorbar=True,)
# plt.legend()

In [ ]:
# 데이터 라벨/숫자변수/카테고리변수로 나누기
df_label = df_source[["MEDV"]]
df_cat = df_source[["CHAS"]]
df_num = df_source.drop(columns=df_cat.columns)
df_num = df_num.drop(columns=df_label.columns)

In [ ]:
# 잘 나눠졌나 확인하기
df_label.columns, df_cat.columns, df_num.columns 

In [ ]:
# row 기준, 결측치 갯수 보기 
df_source.isnull().sum(axis=0)

In [ ]:
# column 기준 결측치가 있는 데이터 보기 
df_source[df_source.isnull().any(axis=1)] 

In [ ]:
# 결측치가 있는 모든 row 삭제 하기 
df_source_nona = df_source.dropna(axis=0)

# 결측치가 있는 모든 column 삭제 하기 
df_source_nona = df_source.dropna(axis=1)

# 특정 Column (MEDV) 에서 결측치 제거 하기 
df_source_nona = df_source.dropna(subset=["MEDV"]) 


In [ ]:
df_source_nona

In [ ]:
#결측치를 제거하고 사용하고 싶을 경우 아래 주석 활성화
#df_source = df_source_nona
#df_cat = df_cat[df_cat.index.isin(df_source.index)]
#df_num = df_num[df_num.index.isin(df_source.index)]
#df_label = df_label[df_label.index.isin(df_source.index)]

In [ ]:
# 특정 조건을 만족하는 데이터만 뽑기 

# 조건을 먼저 선언
# MEDV 열의 값이 5000이상인 경우
condition1 = df_source['MEDV'] > 5000
# MEDV 열의 값이 5000이상이거나 2500 이하인 경우
condition2 = (df_source['MEDV'] > 5000) | (df_source['MEDV'] < 2500)
# MEDV 열의 값이 5000이 아닌경우
condition1 = df_source['MEDV'] != 5000
# 데이터가 string인 경우는 ' ' 


# Condition을 만족하는 값만 남기기
df_new = df_source[condition1]
# Condition을 만족하지 값만 삭제하기
df_new = df_source[~condition1]

# 특정 조건을 만족하는 데이터를 삭제 할 때 인덱스를 이용하는 방법 
idx = df_source[condition2].index
df_new = df_source.drop(idx)

#df_source 자체를 바꾸고 싶으면 inplace=True 를 함수 내에 넣으면 되지만, 웬만하면 df_source 원본은 건드리지 말자!

In [ ]:
from sklearn.impute import SimpleImputer
# SimpleImputer?

In [ ]:
# median 값으로 inputing 하기 
imputer = SimpleImputer(strategy="median") 
# Options: mean, median, most_frequent, constant(eg., fill_value=0)
df_num_imputed = imputer.fit_transform(df_num)
df_num_imputed = pd.DataFrame(df_num_imputed, columns=df_num.columns)
df_num_imputed

In [ ]:
#impute 한 후 결측치가 있는지 확인하기 
df_num_imputed[df_source.isnull().any(axis=1)]

In [ ]:
#impute된 값을 이용하려면 아래 주석 활성화
# df_num = df_num_imputed

In [ ]:
# 아웃라이어 찾기 (특정 조건을 벗어나는 값 찾기)
df_outliers =\
    df_source[(df_source['CHAS']!=0)&(df_source['CHAS']!=1)]
df_outliers

# 아웃라이어 삭제한 데이터 프레임 얻기
df_source_nool = df_source[~df_source.index.isin(df_outliers.index)] # Write your code here.
df_source_nool

In [ ]:
#IRQ 값으로 제거하기 
import numpy as np

def get_outlier(df=None, column=None, weight=1.5):
  quantile_25 = np.percentile(df[column].values, 25)
  quantile_75 = np.percentile(df[column].values, 75)

  IQR = quantile_75 - quantile_25
  IQR_weight = IQR*weight
  
  lowest = quantile_25 - IQR_weight
  highest = quantile_75 + IQR_weight
  
  outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
  return outlier_idx

# 함수 사용해서 'MEDV' column 의 이상치 값 삭제
outlier_idx = get_outlier(df=df_source, column='MEDV', weight=1.5) #weight = 3 이면 3IQR 값으로 진행
df_source_nool = df_source.drop(outlier_idx, axis=0)

In [ ]:
df_source_nool.hist(bins=50, figsize=(20,15)); # Write your code here.

In [ ]:
# 아웃라이어 삭제한 값으로 다시 num/cat/label 나누기 
df_source = df_source_nool
df_num = df_num[df_num.index.isin(df_source.index)] # Write your code here.
df_cat = df_cat[df_cat.index.isin(df_source.index)] # Write your code here.
df_label = df_label[df_label.index.isin(df_source.index)] # Write your code here.

In [ ]:
# 카테고리 데이터 encoding 하기 
# cat_map = {"ISLAND":0, "<1H OCEAN":1, "NEAR OCEAN":2, "NEAR BAY":3, "INLAND":4}
# inv_cat_map = {v: k for k, v in cat_map.items()}
# df_cat_encoded = df_source["ocean_proximity"].map(cat_map)
# df_cat_encoded = pd.DataFrame(df_cat_encoded, columns=df_cat.columns)
# df_cat_encoded

In [ ]:
# type(df_cat_encoded.squeeze())

In [ ]:
# df_cat_encoded.squeeze().map(inv_cat_map)

In [ ]:
#df_cat = df_cat_encoded

In [ ]:
# ordinar 인코딩 아닌경우 아래 활성화 시켜서 일괄 one hot encoding 

# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder(sparse=False)
# df_cat_encoded = encoder.fit_transform(df_cat).astype(int)
# df_cat_encoded = pd.DataFrame(df_cat_encoded, columns=encoder.categories_[0].tolist(), index=df_cat.index) # encoder.categories_ is a list of array
# df_cat_encoded

In [ ]:
# df_cat = df_cat_encoded

In [ ]:
# Feature 추가 
# df_num["rooms_per_household"] = df_num["total_rooms"] / df_num["households"]
# df_num["bedrooms_per_room"] = df_num["total_bedrooms"] / df_num["total_rooms"]
# df_num["population_per_household"] = df_num["population"] / df_num["households"]

In [ ]:
# 추가한 Feature에 대해 correlation 확인
# df_corr = pd.merge(df_num, df_label, left_index=True, right_index=True).corr()
# df_corr["median_house_value"].sort_values(ascending=False)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# PolynomialFeatures?

In [ ]:
# 다항 변수 추가
polyfeat = PolynomialFeatures(degree=2, include_bias=False)
df_num_poly = polyfeat.fit_transform(df_num)
df_num_poly = pd.DataFrame(df_num_poly, columns=polyfeat.get_feature_names_out().tolist(), index=df_num.index)
df_num_poly

In [ ]:
#추가된 변수 이름 보기 
polyfeat.get_feature_names_out()

In [ ]:
#polynomial feature 사용하려면 아래 활성화
# df_num = df_num_poly

In [ ]:
from sklearn.model_selection import train_test_split

# Train/test set 나누기
X_num_train, X_num_test, X_cat_train, X_cat_test, y_train, y_test = \
    train_test_split(df_num, df_cat, df_label, test_size=.2, random_state=42)

# Train/valid set 나누기
X_num_train, X_num_valid, X_cat_train, X_cat_valid, y_train, y_valid = \
    train_test_split(X_num_train, X_cat_train, y_train, test_size=.3, random_state=42)

In [ ]:
# X_train.shape

In [ ]:
# 숫자변수 Scaling 하기 
from sklearn.preprocessing import StandardScaler
# StandardScaler?

In [ ]:
# 숫자변수 Scaling 하기 
scaler = StandardScaler()
X_num_train_scaled = scaler.fit(X_num_train)      # fit() is only for training set!
X_num_train_scaled = scaler.transform(X_num_train)
X_num_valid_scaled = scaler.transform(X_num_valid)
X_num_test_scaled  = scaler.transform(X_num_test)
X_num_train_scaled = pd.DataFrame(X_num_train_scaled, columns=X_num_train.columns, index=X_num_train.index)
X_num_valid_scaled = pd.DataFrame(X_num_valid_scaled, columns=X_num_valid.columns, index=X_num_valid.index)
X_num_test_scaled  = pd.DataFrame(X_num_test_scaled , columns=X_num_test.columns , index=X_num_test.index)
X_num_train_scaled

In [ ]:
# scaled value 사용하려면 아래 활성화
# X_num_train = X_num_train_scaled
# X_num_valid = X_num_valid_scaled
# X_num_test  = X_num_test_scaled

In [ ]:
# scaled value 원복 하기 
# X_num_train = scaler.inverse_transform(X_num_train_scaled)
# X_num_valid = scaler.inverse_transform(X_num_valid_scaled)
# X_num_test  = scaler.inverse_transform(X_num_test_scaled)
# X_num_train = pd.DataFrame(X_num_train, columns=X_num_train_scaled.columns, index=X_num_train_scaled.index)
# X_num_valid = pd.DataFrame(X_num_valid, columns=X_num_valid_scaled.columns, index=X_num_valid_scaled.index)
# X_num_test  = pd.DataFrame(X_num_test , columns=X_num_test_scaled.columns , index=X_num_test_scaled.index)

In [ ]:
# 전처리가 끝났으면 num/cat 변수를 학습데이터로 Merge.
X_train = pd.merge(X_num_train, X_cat_train, left_index=True, right_index=True)
X_valid = pd.merge(X_num_valid, X_cat_valid, left_index=True, right_index=True)
X_test  = pd.merge(X_num_test , X_cat_test , left_index=True, right_index=True)

X_train

In [ ]:
from sklearn.linear_model import LinearRegression
# LinearRegression?

In [ ]:
#linearRegression 기본 모델 학습하기 
# RMSE 값 같이 확인하기

from sklearn.metrics import mean_squared_error
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_train_pred = lin_reg.predict(X_train)
y_valid_pred = lin_reg.predict(X_valid)

rmse_ridge_train = mean_squared_error(y_train, y_train_pred, squared=False) #squared=False 이면 root값으로 return
rmse_ridge_valid = mean_squared_error(y_valid, y_valid_pred, squared=False)

print("Ridge Regression Model's RMSE")
print(rmse_ridge_train)
print(rmse_ridge_valid)
print()
print("Ridge Regression Model's R^2")
print(lin_reg.score(X_train, y_train))
print(lin_reg.score(X_valid, y_valid))

In [ ]:
#Regulation 적용하기 



In [ ]:
# Ridge 기본 모델 학습하기
from sklearn.linear_model import Ridge
# Ridge?

ridge_reg = Ridge(random_state=42) # alpha 기본값은 ??
ridge_reg.fit(X_train, y_train)
print("Ridge Regression Model's R^2")
print(ridge_reg.score(X_train, y_train))
print(ridge_reg.score(X_valid, y_valid))

In [ ]:
# Ridge best alpha 찾기
lst_train_score = []
lst_valid_score = []
lst_alpha = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 
             1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9
            ]
for alpha in lst_alpha:
    # Train a Ridge regression model
    ridge_reg = Ridge(alpha=alpha, random_state=42)
    ridge_reg.fit(X_train, y_train)
    # Store train and validation scores
    lst_train_score.append(ridge_reg.score(X_train, y_train))
    lst_valid_score.append(ridge_reg.score(X_valid, y_valid))

In [ ]:
# alpha 값에 따른 R square값 변화 보기 
import numpy as np

plt.figure(figsize=(10, 8))
plt.plot(np.log10(lst_alpha), lst_train_score, label="training score")
plt.plot(np.log10(lst_alpha), lst_valid_score, label="validation score")
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.legend(loc="lower right", fontsize=14)
plt.show()

In [ ]:
# Best alpha로 학습한값 확인하기
best_alpha = lst_alpha[np.argmax(lst_valid_score)]
print("validation score:", lst_valid_score)
print("best alpha:", best_alpha)

ridge_reg = Ridge(alpha=best_alpha, random_state=42)
ridge_reg.fit(X_train, y_train)

print("R^2 on training-set:", ridge_reg.score(X_train, y_train))
print("R^2 on validation-set:", ridge_reg.score(X_valid, y_valid))


In [ ]:
# Lasso 기본 모델 학습하기
from sklearn.linear_model import Lasso
# Lasso?

lasso_reg = Lasso(random_state=42, max_iter=10000) #iter 기본값??
lasso_reg.fit(X_train, y_train)

y_train_pred = lasso_reg.predict(X_train)
y_valid_pred = lasso_reg.predict(X_valid)

rmse_lasso_train = mean_squared_error(y_train, y_train_pred, squared=False)
rmse_lasso_valid = mean_squared_error(y_valid, y_valid_pred, squared=False)

print("Lasso Regression Model's RMSE")
print(rmse_lasso_train)
print(rmse_lasso_valid)
print()
print("Lasso Regression Model's R^2")
print(lasso_reg.score(X_train, y_train))
print(lasso_reg.score(X_valid, y_valid))

In [ ]:
# Lasso best alpha 찾기
lst_train_score = []
lst_valid_score = []
lst_alpha = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
for alpha in lst_alpha:
    # Train a Lasso regression model
    lasso_reg = Lasso(alpha=alpha, max_iter=1e5, random_state=42)
    lasso_reg.fit(X_train, y_train)
    # Store train and validation scores
    lst_train_score.append(lasso_reg.score(X_train, y_train))
    lst_valid_score.append(lasso_reg.score(X_valid, y_valid))

In [ ]:
# alpha 값에 따른 R square값 변화 보기 
plt.figure(figsize=(10, 8))
plt.plot(np.log10(lst_alpha), lst_train_score, label="training score")
plt.plot(np.log10(lst_alpha), lst_valid_score, label="validation score")
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.legend(loc="lower right", fontsize=14)
plt.show()

In [ ]:
# Best alpha로 학습한값 확인하기
best_alpha = lst_alpha[np.argmax(lst_valid_score)]
print("best alpha:", best_alpha)

lasso_reg = Lasso(alpha=best_alpha, random_state=42)
lasso_reg.fit(X_train, y_train)

print(lasso_reg.score(X_train, y_train))
print(lasso_reg.score(X_valid, y_valid))

In [ ]:
#ElasticNet 기본 모델 학습하기 
from sklearn.linear_model import ElasticNet
# ElasticNet?

elnet_reg = ElasticNet(random_state=42)
elnet_reg.fit(X_train, y_train)

y_train_pred = elnet_reg.predict(X_train)
y_valid_pred = elnet_reg.predict(X_valid)

rmse_elnet_train = mean_squared_error(y_train, y_train_pred, squared=False)
rmse_elnet_valid = mean_squared_error(y_valid, y_valid_pred, squared=False)

print("ElasticNet Regression Model's RMSE")
print(rmse_elnet_train)
print(rmse_elnet_valid)
print()
print("ElasticNet Regression Model's R^2")
print(elnet_reg.score(X_train, y_train))
print(elnet_reg.score(X_valid, y_valid))

In [ ]:
# ElasticNet best alpha 찾기
lst_train_score = []
lst_valid_score = []
lst_alpha = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 
             1e1, 1e2, 1e3, 1e4, 1e5
            ]
for alpha in lst_alpha:
    # Train a Elastic Net model
    elnet_reg = ElasticNet(alpha=alpha, max_iter=1e5, random_state=42)
    elnet_reg.fit(X_train, y_train)
    # Store train and validation scores
    lst_train_score.append(elnet_reg.score(X_train, y_train))
    lst_valid_score.append(elnet_reg.score(X_valid, y_valid))

In [ ]:
# alpha 값에 따른 R square값 변화 보기 
plt.figure(figsize=(10, 8))
plt.plot(np.log10(lst_alpha), lst_train_score, label="training score")
plt.plot(np.log10(lst_alpha), lst_valid_score, label="validation score")
#plt.plot(np.log10(lst_alpha), test_score, label="test score")
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.legend(loc="lower right", fontsize=14)
plt.show()

In [ ]:
# Best alpha로 학습한값 확인하기
best_alpha = lst_alpha[np.argmax(lst_valid_score)]
print("best alpha:", best_alpha)

elnet_reg = ElasticNet(alpha=best_alpha, random_state=42)
elnet_reg.fit(X_train, y_train)

print(elnet_reg.score(X_train, y_train))
print(elnet_reg.score(X_valid, y_valid))

In [ ]:
# 스토케스틱 Regressor 모델 학습하기 
from sklearn.linear_model import SGDRegressor
# SGDRegressor?

In [ ]:
# 규제 없는 SGD plain 모델 학습하기
# verbose = 2 => log 보기 
sgd_plain_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty=None,
                             eta0=1e-4, # 기본값 ??
                             max_iter=1e4, # 기본값 ??
                            ) 

sgd_plain_reg.fit(X_train, y_train.to_numpy().ravel()) 

print("train score:", sgd_plain_reg.score(X_train, y_train.to_numpy().ravel()))
print("validation score:", sgd_plain_reg.score(X_valid, y_valid.to_numpy().ravel()))

In [ ]:
# L2 SGD 모델 학습하기
sgd_ridge_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l2',
                             eta0=1e-4,
                             max_iter=1e4
                            ) 
sgd_ridge_reg.fit(X_train, y_train.to_numpy().ravel()) 

print("train score:", sgd_ridge_reg.score(X_train, y_train))
print("validation score:", sgd_ridge_reg.score(X_valid, y_valid)) 

In [ ]:
# L1 SGD 모델 학습하기
sgd_lasso_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l1',
                             eta0=1e-3,
#                              max_iter=1e4,
                            ) 
sgd_lasso_reg.fit(X_train, y_train.to_numpy().ravel())

print(sgd_lasso_reg.score(X_train, y_train))
print(sgd_lasso_reg.score(X_valid, y_valid))

In [ ]:
# elasticnet SGD 모델 학습하기
sgd_elnet_reg = SGDRegressor(random_state=42,
#                              verbose=2,
                             penalty='elasticnet',
                            #  eta0=1e-3,
#                              max_iter=1e4
                            ) 
sgd_elnet_reg.fit(X_train, y_train.to_numpy().ravel())
# Show the model's scores on training and validation sets.
print("training score:", sgd_elnet_reg.score(X_train, y_train)) 
print("validation score:", sgd_elnet_reg.score(X_valid, y_valid))

# hyperparameter 확인
print(sgd_elnet_reg.penalty, sgd_elnet_reg.alpha, sgd_elnet_reg.l1_ratio)

In [ ]:
# R-Squared on Test-Set
print("Regression Model's Test Score")
print("Linear         :", lin_reg.score(X_test, y_test))
print("Ridge          :", ridge_reg.score(X_test, y_test))
print("Lasso          :", lasso_reg.score(X_test, y_test))
print("Elastic Net    :", elnet_reg.score(X_test, y_test))
print("SGD Plain      :", sgd_plain_reg.score(X_test, y_test))
print("SGD Ridge      :", sgd_ridge_reg.score(X_test, y_test))
print("SGD Lasso      :", sgd_lasso_reg.score(X_test, y_test))
print("SGD Elastic Net:", sgd_elnet_reg.score(X_test, y_test))

In [ ]:
# (Grid) cross-validation 을 이용한 Fine tuning 
# left_index and right_index=True should be included 
# cross validation 은 여러 셋으로 validation 함으로 test 셋을 나눌 필요 없음
if len(pd.merge(X_train, X_valid, left_index=True, right_index=True)) == 0:
    X_train, y_train = pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid])

In [ ]:
from sklearn.model_selection import GridSearchCV
# GridSearchCV?

In [ ]:
# Ridge 모델 Fine tuning with grid search  cross validation 
from sklearn.linear_model import Ridge
ridge_reg = Ridge(random_state=42)

params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2]}
ridge_grid_cv = GridSearchCV(estimator=ridge_reg, 
                             param_grid=params, 
                             return_train_score=True,
                             n_jobs=-1,
                            )
ridge_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print(ridge_grid_cv.best_score_)
print(ridge_grid_cv.best_params_)
ridge_reg = ridge_grid_cv.best_estimator_

In [ ]:
ridge_grid_cv_results = pd.DataFrame(ridge_grid_cv.cv_results_)
# 결과를 파일로 저장하고 싶으면 활성화
# ridge_grid_cv_results.to_csv("ridge_grid_cv_results.csv")
ridge_grid_cv_results

In [ ]:
# np.linspace?
# Grid search 항목인 alpha에 linspace 이용
from sklearn.linear_model import Ridge
ridge_reg = Ridge(random_state=42)

params = {'alpha': np.linspace(1e0, 1e2, 100)} # log 함수랑 달리 동일한 간격으로 100개 추출
ridge_grid_cv = GridSearchCV(estimator=ridge_reg, 
                             param_grid=params, 
                             return_train_score=True,
                             n_jobs=-1,
                            ) 
ridge_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", ridge_grid_cv.best_score_)
print("Best Parameters:", ridge_grid_cv.best_params_)
ridge_reg = ridge_grid_cv.best_estimator_

In [ ]:
# Lasso 모델 Gridsearch로 fine tuning
from sklearn.linear_model import Lasso
lasso_reg = Lasso(random_state=42,
                  max_iter=1e7,
                 )

# params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2]}
params = {'alpha': [1e1, 1e2, 1e3]}
lasso_grid_cv = GridSearchCV(estimator=lasso_reg, 
                             param_grid=params, 
                             return_train_score=True,
                             n_jobs=-1,
                            )
lasso_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", lasso_grid_cv.best_score_)
print("Best Parameters:", lasso_grid_cv.best_params_)
lasso_reg = lasso_grid_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha값의 range설정 
from sklearn.linear_model import Lasso
lasso_reg = Lasso(random_state=42,
                  max_iter=1e7,
                 )

# params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2]}
params = {'alpha': np.linspace(1e1, 1e3, 100)}
lasso_grid_cv = GridSearchCV(estimator=lasso_reg, 
                             param_grid=params, 
                             return_train_score=True,
                             n_jobs=-1,
                            )
lasso_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", lasso_grid_cv.best_score_)
print("Best Parameters:", lasso_grid_cv.best_params_)
lasso_reg = lasso_grid_cv.best_estimator_

In [ ]:
# ElasticNet 모델 Gridsearch로 fine tuning
from sklearn.linear_model import ElasticNet
elnet_reg = ElasticNet(random_state=42,
                       max_iter=1e4,
                      )
# params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2],
#           'l1_ratio': [.1, .2, .3, .4, .5, .6, .7, .8, .9],
#          }
params = {'alpha': [1e-3, 1e-2, 1e-1],
          'l1_ratio': [.1, .2, .3, .4, .5, .6, .7, .8, .9],
         }
elnet_grid_cv = GridSearchCV(estimator=elnet_reg, 
                             param_grid=params, 
                             return_train_score=True,
                             n_jobs=-1,
                            )
elnet_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", elnet_grid_cv.best_score_)
print("Best Parameters:", elnet_grid_cv.best_params_)
elnet_reg = elnet_grid_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha값의 range설정
from sklearn.linear_model import ElasticNet
elnet_reg = ElasticNet(random_state=42,
                       max_iter=1e4,
                      )
params = {'alpha': np.linspace(1e-3, 1e-1, 100),
          'l1_ratio': np.linspace(.7, .9, 10),
         }
elnet_grid_cv = GridSearchCV(estimator=elnet_reg, 
                             param_grid=params, 
                             return_train_score=True,
                             n_jobs=-1,
                            )
elnet_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", elnet_grid_cv.best_score_)
print("Best Parameters:", elnet_grid_cv.best_params_)
elnet_reg = elnet_grid_cv.best_estimator_

In [ ]:
# SGD 모델 fine tuning
from sklearn.linear_model import SGDRegressor
sgd_plain_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty=None,
                             eta0=1e-4,
                             max_iter=1e4,
                            )
params = {}
sgd_plain_grid_cv = GridSearchCV(estimator=sgd_plain_reg,
                                 param_grid=params,
                                 return_train_score=True,
                                 n_jobs=-1,
                                )
sgd_plain_grid_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_plain_grid_cv.best_score_)
print("Best Parameters:", sgd_plain_grid_cv.best_params_)
sgd_plain_reg = sgd_plain_grid_cv.best_estimator_

In [ ]:
# SGD Ridge 모델 fine tuning
sgd_ridge_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l2',
                             eta0=1e-4,
                             max_iter=1e4,
                            )
# params = {'alpha':[1e-2, 1e-1, 1e0, 1e1, 1e2]}
params = {'alpha':[1e-4, 1e-3, 1e-2, 1e-1]}
sgd_ridge_grid_cv = GridSearchCV(estimator=sgd_ridge_reg,
                                 param_grid=params,
                                 return_train_score=True,
                                 n_jobs=-1,
                                )
sgd_ridge_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_ridge_grid_cv.best_score_)
print("Best Parameters:", sgd_ridge_grid_cv.best_params_)
sgd_ridge_reg = sgd_ridge_grid_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha값의 range설정
sgd_ridge_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l2',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {'alpha':np.linspace(1e-4, 1e-2, 100)}
sgd_ridge_grid_cv = GridSearchCV(estimator=sgd_ridge_reg,
                                 param_grid=params,
                                 return_train_score=True,
                                 n_jobs=-1,
                                )
sgd_ridge_grid_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_ridge_grid_cv.best_score_)
print("Best Parameters:", sgd_ridge_grid_cv.best_params_)
sgd_ridge_reg = sgd_ridge_grid_cv.best_estimator_

In [ ]:
# SGD Lasso 모델 fine tuning
from sklearn.linear_model import SGDRegressor
sgd_lasso_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l1',
                             eta0=1e-4,
                             max_iter=1e4
                            ) # Write your code here
params = {'alpha':[1e-2, 1e-1, 1e0, 1e1, 1e2]}
sgd_lasso_grid_cv = GridSearchCV(estimator=sgd_lasso_reg,
                                 param_grid=params,
                                 return_train_score=True,
                                 n_jobs=-1,
                                )
sgd_lasso_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_lasso_grid_cv.best_score_)
print("Best Parameters:", sgd_lasso_grid_cv.best_params_)
sgd_lasso_reg = sgd_lasso_grid_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha값의 range설정

from sklearn.linear_model import SGDRegressor
sgd_lasso_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l1',
                             eta0=1e-4,
                             max_iter=1e4
                            ) # Write your code here
params = {'alpha': np.linspace(1e0, 1e2, 100)}
sgd_lasso_grid_cv = GridSearchCV(estimator=sgd_lasso_reg,
                                 param_grid=params,
                                 return_train_score=True,
                                 n_jobs=-1,
                                )
sgd_lasso_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_lasso_grid_cv.best_score_)
print("Best Parameters:", sgd_lasso_grid_cv.best_params_)
sgd_lasso_reg = sgd_lasso_grid_cv.best_estimator_

In [ ]:
# SGD elasticnet 모델 fine tuning
# l1_ratio에 0.0, 1.0을 포함하면 Ridge, Lasso를 굳이 안돌려도 됨. 
# 근데 시험에서는 그래도 되는지 모르겠음. 
from sklearn.linear_model import SGDRegressor
sgd_elnet_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='elasticnet',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2],
          'l1_ratio': [.1, .2, .3, .4, .5, .6, .7, .8, .9],
         }
sgd_elnet_grid_cv = GridSearchCV(estimator=sgd_elnet_reg,
                                 param_grid=params,
                                 return_train_score=True,
                                 n_jobs=-1,
                                )
sgd_elnet_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_elnet_grid_cv.best_score_)
print("Best Parameters:", sgd_elnet_grid_cv.best_params_)
sgd_elnet_reg = sgd_elnet_grid_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha, L1_ratio값의 range설정

from sklearn.linear_model import SGDRegressor
sgd_elnet_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='elasticnet',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {'alpha': np.linspace(1e-3, 1e-1, 10),
          'l1_ratio': np.linspace(.7, .9, 10),
         }
sgd_elnet_grid_cv = GridSearchCV(estimator=sgd_elnet_reg,
                                 param_grid=params,
                                 return_train_score=True,
                                 n_jobs=-1,
                                )
sgd_elnet_grid_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_elnet_grid_cv.best_score_)
print("Best Parameters:", sgd_elnet_grid_cv.best_params_)
sgd_elnet_reg = sgd_elnet_grid_cv.best_estimator_

In [ ]:
# R-Squared on Test-Set
print("Regression Model's Test Score")
print("Linear         :", lin_reg.score(X_test, y_test))
print("Ridge          :", ridge_reg.score(X_test, y_test))
print("Lasso          :", lasso_reg.score(X_test, y_test))
print("Elastic Net    :", elnet_reg.score(X_test, y_test))
print("SGD Ridge      :", sgd_ridge_reg.score(X_test, y_test))
print("SGD Lasso      :", sgd_lasso_reg.score(X_test, y_test))
print("SGD Elastic Net:", sgd_elnet_reg.score(X_test, y_test))

In [ ]:
# Ridge 모델 Fine tuning with randomized search cross validation 
from sklearn.model_selection import RandomizedSearchCV
# RandomizedSearchCV?
from scipy.stats import uniform, randint
# uniform
# https://rfriend.tistory.com/284
# 균등분포 무작위 샘플 추출

In [ ]:
# Cross Validation 초반에 train 데이터 합치는 부분 안했으면 아래 부분 활성화해서 다시 수행
# if len(pd.merge(X_train, X_valid)) == 0:
#     X_train, y_train = pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid])

In [ ]:
# Fine-Tune Ridge Models with randomized search cross validation 
from sklearn.linear_model import Ridge
ridge_reg = Ridge(random_state=42)

# params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2]}
params = {'alpha': uniform(1e-2, 1e2)}
ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=ridge_reg, 
                                   param_distributions=params, 
                                   n_iter=10,
                                   return_train_score=True,
                                   n_jobs=-1,
                                  ) # Write your code here.
ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(ridge_rand_cv.best_score_)
print(ridge_rand_cv.best_params_)
ridge_reg = ridge_rand_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha값의 range설정
from sklearn.linear_model import Ridge
ridge_reg = Ridge(random_state=42)

# params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2]}
params = {'alpha': uniform(1e0, 1e2)}
ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=ridge_reg, 
                                   param_distributions=params, 
                                   n_iter=100,
                                   return_train_score=True,
                                   n_jobs=-1,
                                  ) # Write your code here.
ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(ridge_rand_cv.best_score_)
print(ridge_rand_cv.best_params_)
ridge_reg = ridge_rand_cv.best_estimator_

In [ ]:
# Fine-Tune Lasso Models with randomized search cross validation 
from sklearn.linear_model import Lasso
lasso_reg = Lasso(random_state=42,
                  max_iter=1e7,
                 ) # Write your code here.

# params = {'alpha': uniform(1e-2, 1e2)}
params = {'alpha': uniform(1e1, 1e3)}
lasso_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=lasso_reg, 
                                   param_distributions=params, 
                                   return_train_score=True,
                                   n_iter=10,
                                   n_jobs=-1,
                                  )
lasso_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(lasso_rand_cv.best_score_)
print(lasso_rand_cv.best_params_)
lasso_reg = lasso_rand_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha값의 range설정
from sklearn.linear_model import Lasso
lasso_reg = Lasso(random_state=42,
                  max_iter=1e7,
                 ) # Write your code here.

params = {'alpha': uniform(1e1, 1e2)}
lasso_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=lasso_reg, 
                                   param_distributions=params, 
                                   return_train_score=True,
                                   n_iter=100,
                                   n_jobs=-1,
                                  )
lasso_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(lasso_rand_cv.best_score_)
print(lasso_rand_cv.best_params_)
lasso_reg = lasso_rand_cv.best_estimator_

In [ ]:
# Fine-Tune ElasticNet Models with randomized search cross validation 
from sklearn.linear_model import ElasticNet
elnet_reg = ElasticNet(random_state=42,
                       max_iter=1e4,
                      ) # Write your code here.
params = {'alpha': uniform(1e-2, 1e2),
          'l1_ratio': uniform(0, 1),
         }
elnet_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=elnet_reg, 
                                   param_distributions=params, 
                                   return_train_score=True, 
                                   n_iter=10,
                                   n_jobs=-1,
                                  ) # Write your code here.
elnet_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(elnet_rand_cv.best_score_)
print(elnet_rand_cv.best_params_)
elnet_reg = elnet_rand_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha, l1_ratio값의 range설정
from sklearn.linear_model import ElasticNet
elnet_reg = ElasticNet(random_state=42,
                       max_iter=1e4,
                      ) # Write your code here.
params = {'alpha': uniform(1e-1, 1e1),
          'l1_ratio': uniform(0.0, 1.0),
         }
elnet_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=elnet_reg, 
                                   param_distributions=params, 
                                   return_train_score=True, 
                                   n_iter=100,
                                   n_jobs=-1,
                                  ) # Write your code here.
elnet_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(elnet_rand_cv.best_score_)
print(elnet_rand_cv.best_params_)
elnet_reg = elnet_rand_cv.best_estimator_

In [ ]:
# Fine-Tune SGD Plain Models with randomized search cross validation
from sklearn.linear_model import SGDRegressor
sgd_plain_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty=None,
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {}
sgd_plain_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_plain_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10,
                                      )
sgd_plain_rand_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_plain_rand_cv.best_score_)
print("Best Parameters:", sgd_plain_rand_cv.best_params_)
sgd_plain_reg = sgd_plain_rand_cv.best_estimator_


In [ ]:

print(pd.DataFrame(sgd_plain_rand_cv.cv_results_))

In [ ]:
# Fine-Tune SGD Ridge Models with randomized search cross validation
sgd_ridge_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l2',
                            ) # Write your code here
params = {'alpha': uniform(1e-2, 1e2)}
# params = {'alpha': uniform(1e-3, 1e3)}
# params = {'alpha': uniform(1e-4, 1e4)}
sgd_ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_ridge_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10,
#                                        n_iter=100,
#                                        n_iter=1000,
#                                        n_iter=10000,
                                      )
sgd_ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_ridge_rand_cv.best_score_)
print("Best Parameters:", sgd_ridge_rand_cv.best_params_)
sgd_ridge_reg = sgd_ridge_rand_cv.best_estimator_

In [ ]:
#다른 방식으로도 찾아보기
from sklearn.linear_model import SGDRegressor
sgd_ridge_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l2',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
# params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2]}
params = {'alpha': [1e-4, 1e-3, 1e-2, 1e-1]}
sgd_ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_ridge_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10
                                      )
sgd_ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_ridge_rand_cv.best_score_)
print("Best Parameters:", sgd_ridge_rand_cv.best_params_)
sgd_ridge_reg = sgd_ridge_rand_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha, l1_ratio값의 range설정
from sklearn.linear_model import SGDRegressor
sgd_ridge_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l2',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {'alpha': uniform(1e-3, 1e-1)}
sgd_ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_ridge_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=100
                                      )
sgd_ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_ridge_rand_cv.best_score_)
print("Best Parameters:", sgd_ridge_rand_cv.best_params_)
sgd_ridge_reg = sgd_ridge_rand_cv.best_estimator_

In [ ]:
# Fine-Tune SGD Lasso Models with randomized search cross validation
from sklearn.linear_model import SGDRegressor
sgd_lasso_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l1',
                             eta0=1e-4,
                             max_iter=1e4
                            ) # Write your code here
params = {'alpha':[1e-2, 1e-1, 1e0, 1e1, 1e2]}
sgd_lasso_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_lasso_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10,
                                      )
sgd_lasso_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_lasso_rand_cv.best_score_)
print("Best Parameters:", sgd_lasso_rand_cv.best_params_)
sgd_lasso_reg = sgd_lasso_rand_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha, l1_ratio값의 range설정
from sklearn.linear_model import SGDRegressor
sgd_lasso_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l1',
                             eta0=1e-4,
                             max_iter=1e4
                            ) # Write your code here
params = {'alpha': uniform(1e0, 1e2)}
sgd_lasso_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_lasso_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=100,
                                      )
sgd_lasso_rand_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_lasso_rand_cv.best_score_)
print("Best Parameters:", sgd_lasso_rand_cv.best_params_)
sgd_lasso_reg = sgd_lasso_rand_cv.best_estimator_

In [ ]:
# Fine-Tune SGD Elnet Models with randomized search cross validation
from sklearn.linear_model import SGDRegressor
sgd_elnet_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='elasticnet',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {'alpha':[1e-2, 1e-1, 1e0, 1e1, 1e2]}
sgd_elnet_rand_cv = RandomizedSearchCV(random_state=42,
                                       estimator=sgd_elnet_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10,
                                      )
sgd_elnet_rand_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_elnet_rand_cv.best_score_)
print("Best Parameters:", sgd_elnet_rand_cv.best_params_)
sgd_elnet_reg = sgd_elnet_rand_cv.best_estimator_

In [ ]:
#위에서 찾은 값을 기준으로 더 세분화 하여 alpha, l1_ratio값의 range설정
from sklearn.linear_model import SGDRegressor
sgd_elnet_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='elasticnet',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {'alpha': uniform(1e-4, 1e-2)}
sgd_elnet_rand_cv = RandomizedSearchCV(random_state=42,
                                       estimator=sgd_elnet_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=100,
                                      )
sgd_elnet_rand_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_elnet_rand_cv.best_score_)
print("Best Parameters:", sgd_elnet_rand_cv.best_params_)
sgd_elnet_reg = sgd_elnet_rand_cv.best_estimator_

In [ ]:
# R-Squared on Test-Set
print("Regression Model's Test Score")
print("Linear         :", lin_reg.score(X_test, y_test))
print("Ridge          :", ridge_reg.score(X_test, y_test))
print("Lasso          :", lasso_reg.score(X_test, y_test))
print("Elastic Net    :", elnet_reg.score(X_test, y_test))
print("SGD Ridge      :", sgd_ridge_reg.score(X_test, y_test))
print("SGD Lasso      :", sgd_lasso_reg.score(X_test, y_test))
print("SGD Elastic Net:", sgd_elnet_reg.score(X_test, y_test))

In [ ]:
# Incremental learning
# plain 모델을 재학습하여 정확도를 확인
import copy
sgd_partial_reg = copy.deepcopy(sgd_plain_reg)

X_partial_train, X_partial_valid, y_partial_train, y_partial_valid = \
    train_test_split(X_test, y_test, test_size=.2, random_state=42)

In [ ]:
sgd_partial_reg = copy.deepcopy(sgd_plain_reg)
best_sgd_partial_reg = copy.deepcopy(sgd_plain_reg)

lst_epoch = range(0, 100000)
lst_train_score = []
lst_valid_score = []

best_valid_score = 0
no_improvement_cnt = 0
for epoch in lst_epoch:
    # Train the model
    sgd_partial_reg.partial_fit(X_partial_train, y_partial_train.to_numpy().ravel())
    # Store training and validation scores
    train_score = sgd_partial_reg.score(X_partial_train, y_partial_train) 
    valid_score = sgd_partial_reg.score(X_partial_valid, y_partial_valid) 
    lst_train_score.append(train_score)
    lst_valid_score.append(valid_score)
    no_improvement_cnt += 1
    if valid_score > best_valid_score:
        best_valid_score = valid_score
        best_epoch = epoch
        best_sgd_partial_reg = copy.deepcopy(sgd_partial_reg)
        n_not_enhanced = 0
#     if no_improvement_cnt > 100:
#         break

print("best epoch:", best_epoch)
print("training score:", sgd_partial_reg.score(X_partial_train, y_partial_train))
print("validation score:", sgd_partial_reg.score(X_partial_valid, y_partial_valid))

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 8))
plt.plot(lst_train_score, label="training")
plt.plot(lst_valid_score, label="validation")
plt.xlabel('epoch')
plt.ylabel('R^2')
plt.legend(loc="lower right", fontsize=14)
# plt.xlim(0,100)
plt.ylim(0.5,0.7)
plt.show()

In [ ]:
print("R^2 before incremental learning:", sgd_plain_reg.score(X_partial_valid, y_partial_valid)) 
print("R^2 after incremental learning:", sgd_partial_reg.score(X_partial_valid, y_partial_valid)) 

In [ ]:
# Linear SVR 모델 학습하기
from sklearn.svm import LinearSVR
# LinearSVR?
# Check the error caused by the code below.
lsvr_reg = LinearSVR(random_state=42,
                     verbose=0,
                    )
lsvr_reg.fit(X_train, y_train.to_numpy().ravel())
print("Linear SVR Model's Accuracy")
print("Training Set:", lsvr_reg.score(X_train, y_train))
print("Validation Set:", lsvr_reg.score(X_valid, y_valid))

In [ ]:
# SVR 모델 학습하기
from sklearn.svm import SVR
# SVR?
# Check the error caused by the code below.
svr_reg = SVR(verbose=False)
svr_reg.fit(X_train, y_train.to_numpy().ravel())
print("SVR Model's Accuracy")
print("Training Set:", svr_reg.score(X_train, y_train))
print("Validation Set:", svr_reg.score(X_valid, y_valid))

In [ ]:
# # Train an SGDRegressor model with no penalty (None), random_state of 42, and verbose of 2.
# sgd_plain_reg = SGDRegressor(random_state=42,
#                              verbose=0,
#                              penalty=None,
#                              eta0=1e-4,
#                              max_iter=1e4,
#                             ) # Write your code here
# sgd_plain_reg.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
# # Show the model's scores on training and validation sets.
# print("train score:", sgd_plain_reg.score(X_train, y_train.to_numpy().ravel())) # Write your code here
# print("validation score:", sgd_plain_reg.score(X_valid, y_valid.to_numpy().ravel())) # Write your code here

In [ ]:
# Train an SGDRegressor model with no penalty (None), random_state of 42, and verbose of 2.
sgd1_svm_reg = SGDRegressor(random_state=42,
                           verbose=0,
                           loss='epsilon_insensitive', # loss for SVM
                           penalty='l1', # plain model
                           max_iter=1e7, # lower iteration causes convergence wanining 
                           learning_rate='adaptive', # 'invscaling' (default) is too slow
                          )
sgd1_svm_reg.fit(X_train, y_train.to_numpy().ravel())
# Show the model's scores on training and validation sets.
print("train score:", sgd1_svm_reg.score(X_train, y_train))
print("validation score:", sgd1_svm_reg.score(X_valid, y_valid))

In [ ]:
# R-Squared on Test-Set
print("Regression Model's Test Score")
print("Linear SVR       :", lsvr_reg.score(X_test, y_test))
print("SVR              :", svr_reg.score(X_test, y_test))
print("SVM SGD Regressor:", sgd1_svm_reg.score(X_test, y_test))

## <font color=blue>[9] Neural Network Models

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def build_model(n_hidden=1, n_neurons=30, input_shape=[13]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    return model

In [ ]:
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.optimizers import SGD, Adam, Adagrad
from keras.callbacks import EarlyStopping
KerasRegressor?

In [ ]:
nn_reg = \
    KerasRegressor(
        model=build_model,
        random_state=42,
        optimizer=SGD,
        loss="mse",
    )
nn_reg.fit(
    X_train, y_train, 
    validation_data=(X_valid, y_valid),
    callbacks=[EarlyStopping(patience=10)],
)
print(nn_reg.score(X_valid, y_valid))
display(nn_reg.model)

In [ ]:
nn_reg = \
    KerasRegressor(
        model=build_model,
        random_state=42,
        optimizer=SGD,
        loss="mse",
        epochs=100,
        batch_size=16,
    )
nn_reg.fit(
    X_train, y_train, 
    validation_data=(X_valid, y_valid),
    callbacks=[EarlyStopping(patience=10)],
)
print(nn_reg.score(X_valid, y_valid))
display(nn_reg.model)

# <font color=black>[2] Test Models (Option 0)

In [ ]:
# R-Squared on Test-Set
print("Regression Model's Test Score")
print("Linear         :", lin_reg.score(X_test, y_test))
print("Ridge          :", ridge_reg.score(X_test, y_test))
print("Lasso          :", lasso_reg.score(X_test, y_test))
print("Elastic Net    :", elnet_reg.score(X_test, y_test))
print("SGD Ridge      :", sgd_ridge_reg.score(X_test, y_test))
print("SGD Lasso      :", sgd_lasso_reg.score(X_test, y_test))
print("SGD Elastic Net:", sgd_elnet_reg.score(X_test, y_test))
print("Neural Network :", nn_reg.score(X_test, y_test))

# <font color=blue>[5] Fine-Tune Models with Radomized Search Cross-Validation (Option 2)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
RandomizedSearchCV?

In [ ]:
from scipy.stats import randint, uniform, loguniform

In [ ]:
if len(pd.merge(X_train, X_valid, left_index=True, right_index=True)) == 0:
    X_train, y_train = pd.concat([X_train, X_valid]), pd.concat([y_train, y_valid])

## <font color=blue>[3] Fine-Tune Regularized Models

### <font color=blue>[5] Fine-Tune Ridge Models

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(random_state=42)

# params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2]}
params = {'alpha': uniform(1e-2, 1e2)}
ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=ridge_reg, 
                                   param_distributions=params, 
                                   n_iter=10,
                                   return_train_score=True,
                                   n_jobs=-1,
                                  ) # Write your code here.
ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(ridge_rand_cv.best_score_)
print(ridge_rand_cv.best_params_)
ridge_reg = ridge_rand_cv.best_estimator_

Let's reduce the search range and spacing.

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(random_state=42)

# params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2]}
params = {'alpha': uniform(1e0, 1e2)}
ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=ridge_reg, 
                                   param_distributions=params, 
                                   n_iter=100,
                                   return_train_score=True,
                                   n_jobs=-1,
                                  ) # Write your code here.
ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(ridge_rand_cv.best_score_)
print(ridge_rand_cv.best_params_)
ridge_reg = ridge_rand_cv.best_estimator_

### <font color=blue>[5] Fine-Tune Lasso Models

In [ ]:
# Write your code here.
from sklearn.linear_model import Lasso
lasso_reg = Lasso(random_state=42,
                  max_iter=10000000,
                 ) # Write your code here.

# params = {'alpha': uniform(1e-2, 1e2)}
params = {'alpha': uniform(1e-4, 1e-2)}
lasso_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=lasso_reg, 
                                   param_distributions=params, 
                                   return_train_score=True,
                                   n_iter=10,
                                   n_jobs=-1,
                                  )
lasso_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(lasso_rand_cv.best_score_)
print(lasso_rand_cv.best_params_)
lasso_reg = lasso_rand_cv.best_estimator_

In [ ]:
# Write your code here.
from sklearn.linear_model import Lasso
lasso_reg = Lasso(random_state=42,
                  max_iter=10000000,
                 ) # Write your code here.

params = {'alpha': uniform(1e-4, 1e-2)}
lasso_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=lasso_reg, 
                                   param_distributions=params, 
                                   return_train_score=True,
                                   n_iter=100,
                                   n_jobs=-1,
                                  )
lasso_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(lasso_rand_cv.best_score_)
print(lasso_rand_cv.best_params_)
lasso_reg = lasso_rand_cv.best_estimator_

### <font color=blue>[5] Fine-Tune Elastic Net Models

In [ ]:
# Write your code here.
from sklearn.linear_model import ElasticNet
elnet_reg = ElasticNet(random_state=42,
                       max_iter=10000,
                      ) # Write your code here.
params = {'alpha': uniform(1e-2, 1e2),
          'l1_ratio': uniform(0, 1),
         }
elnet_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=elnet_reg, 
                                   param_distributions=params, 
                                   return_train_score=True, 
                                   n_iter=10,
                                   n_jobs=-1,
                                  ) # Write your code here.
elnet_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(elnet_rand_cv.best_score_)
print(elnet_rand_cv.best_params_)
elnet_reg = elnet_rand_cv.best_estimator_

In [ ]:
# Write your code here.
from sklearn.linear_model import ElasticNet
elnet_reg = ElasticNet(random_state=42,
                       max_iter=10000,
                      ) # Write your code here.
params = {'alpha': uniform(1e-1, 1e1),
          'l1_ratio': uniform(0.0, 1.0),
         }
elnet_rand_cv = RandomizedSearchCV(random_state=42, 
                                   estimator=elnet_reg, 
                                   param_distributions=params, 
                                   return_train_score=True, 
                                   n_iter=100,
                                   n_jobs=-1,
                                  ) # Write your code here.
elnet_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print(elnet_rand_cv.best_score_)
print(elnet_rand_cv.best_params_)
elnet_reg = elnet_rand_cv.best_estimator_

## <font color=blue>[5] Fine-Tune SGD Models

### <font color=blue>[5] Fine-Tune SGD Plain Models

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_plain_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty=None,
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {}
sgd_plain_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_plain_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10,
                                      )
sgd_plain_rand_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_plain_rand_cv.best_score_)
print("Best Parameters:", sgd_plain_rand_cv.best_params_)
sgd_plain_reg = sgd_plain_rand_cv.best_estimator_

In [ ]:
pd.DataFrame(sgd_plain_rand_cv.cv_results_)

### <font color=blue>[5] Fine-Tune SGD Ridge Models

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_ridge_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l2',
                            ) # Write your code here
params = {'alpha': uniform(1e-2, 1e2)}
# params = {'alpha': uniform(1e-3, 1e3)}
# params = {'alpha': uniform(1e-4, 1e4)}
sgd_ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_ridge_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10,
#                                        n_iter=100,
#                                        n_iter=1000,
#                                        n_iter=10000,
                                      )
sgd_ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_ridge_rand_cv.best_score_)
print("Best Parameters:", sgd_ridge_rand_cv.best_params_)
sgd_ridge_reg = sgd_ridge_rand_cv.best_estimator_

Let's try a different way.

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_ridge_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l2',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
# params = {'alpha': [1e-2, 1e-1, 1e0, 1e1, 1e2]}
params = {'alpha': [1e-4, 1e-3, 1e-2, 1e-1]}
sgd_ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_ridge_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10
                                      )
sgd_ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_ridge_rand_cv.best_score_)
print("Best Parameters:", sgd_ridge_rand_cv.best_params_)
sgd_ridge_reg = sgd_ridge_rand_cv.best_estimator_

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_ridge_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l2',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {'alpha': uniform(1e-3, 1e-1)}
sgd_ridge_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_ridge_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=100
                                      )
sgd_ridge_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_ridge_rand_cv.best_score_)
print("Best Parameters:", sgd_ridge_rand_cv.best_params_)
sgd_ridge_reg = sgd_ridge_rand_cv.best_estimator_

### <font color=blue>[5] Fine-Tune SGD Lasso Models

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_lasso_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l1',
                             eta0=1e-4,
                             max_iter=1e4
                            ) # Write your code here
params = {'alpha':[1e-2, 1e-1, 1e0, 1e1, 1e2]}
sgd_lasso_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_lasso_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10,
                                      )
sgd_lasso_rand_cv.fit(X_train, y_train.to_numpy().ravel())
print("Best Score:", sgd_lasso_rand_cv.best_score_)
print("Best Parameters:", sgd_lasso_rand_cv.best_params_)
sgd_lasso_reg = sgd_lasso_rand_cv.best_estimator_

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_lasso_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='l1',
                             eta0=1e-4,
                             max_iter=1e4
                            ) # Write your code here
params = {'alpha': uniform(1e0, 1e2)}
sgd_lasso_rand_cv = RandomizedSearchCV(random_state=42, 
                                       estimator=sgd_lasso_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=100,
                                      )
sgd_lasso_rand_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_lasso_rand_cv.best_score_)
print("Best Parameters:", sgd_lasso_rand_cv.best_params_)
sgd_lasso_reg = sgd_lasso_rand_cv.best_estimator_

### <font color=blue>[5] Fine-Tune SGD Elastic Net Models

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_elnet_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='elasticnet',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {'alpha':[1e-2, 1e-1, 1e0, 1e1, 1e2]}
sgd_elnet_rand_cv = RandomizedSearchCV(random_state=42,
                                       estimator=sgd_elnet_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=10,
                                      )
sgd_elnet_rand_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_elnet_rand_cv.best_score_)
print("Best Parameters:", sgd_elnet_rand_cv.best_params_)
sgd_elnet_reg = sgd_elnet_rand_cv.best_estimator_

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_elnet_reg = SGDRegressor(random_state=42,
                             verbose=0,
                             penalty='elasticnet',
                             eta0=1e-4,
                             max_iter=1e4,
                            ) # Write your code here
params = {'alpha': uniform(1e-4, 1e-2)}
sgd_elnet_rand_cv = RandomizedSearchCV(random_state=42,
                                       estimator=sgd_elnet_reg,
                                       param_distributions=params,
                                       return_train_score=True,
                                       n_jobs=-1,
                                       n_iter=100,
                                      )
sgd_elnet_rand_cv.fit(X_train, y_train.to_numpy().ravel()) # Write your code here
print("Best Score:", sgd_elnet_rand_cv.best_score_)
print("Best Parameters:", sgd_elnet_rand_cv.best_params_)
sgd_elnet_reg = sgd_elnet_rand_cv.best_estimator_

## <font color=blue>[10] Fine-tune Neural Network Models

In [ ]:
def build_model(n_hidden=1, n_neurons=30, activation="relu", input_shape=[13]):
    model = # Write your code here
    model.add(# Write your code here)
    for layer in range(# Write your code here):
        model.add(# Write your code here)
    model.add(# Write your code here)
    return model

In [ ]:
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.optimizers import SGD, Adam, Adagrad
from keras.callbacks import EarlyStopping
KerasRegressor?

In [ ]:
pd.DataFrame(nn_rand_cv.cv_results_).to_csv("nn_rand_cv.cv_results_.csv")

In [ ]:
from scipy.stats import loguniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.optimizers import SGD, Adam, RMSprop, Adagrad

nn_reg = KerasRegressor(# Write your code here
                       )
params = # Write your code here
nn_rand_cv = RandomizedSearchCV(# Write your code here
                               )
nn_rand_cv.fit(# Write your code here
              )
# nn_reg = nn_rand_cv.best_estimator_
# print("Best Score:", nn_rand_cv.best_score_)
# print("Best Params:", nn_rand_cv.best_params_)

In [ ]:
nn_reg = nn_rand_cv.best_estimator_
print("Best Score:", nn_rand_cv.best_score_)
print("Best Params:", nn_rand_cv.best_params_)

# <font color=black>[2] Test Models (Option 2)

In [ ]:
# R-Squared on Test-Set
print("Regression Model's Test Score")
print("Linear         :", lin_reg.score(X_test, y_test))
print("Ridge          :", ridge_reg.score(X_test, y_test))
print("Lasso          :", lasso_reg.score(X_test, y_test))
print("Elastic Net    :", elnet_reg.score(X_test, y_test))
print("SGD Ridge      :", sgd_ridge_reg.score(X_test, y_test))
print("SGD Lasso      :", sgd_lasso_reg.score(X_test, y_test))
print("SGD Elastic Net:", sgd_elnet_reg.score(X_test, y_test))
print("Neural Network :", nn_reg.score(X_test, y_test))

Let's compare the scores with the previous ones.

# <font color=black> Supplements

# <font color=black>Exercises

## <font color=blue>[10] Enter appropriate codes where the comments say "Write your code here".

## <font color=blue>[10] Train models with polynomial features and compare their performance with each other.

# References

## <font color=blue>How to Install TensorFlow on an M1 MacBook

## <font color=blue>SciKeras

Follow the "Users Installation" here:<br>
https://www.adriangb.com/scikeras/stable/install.html

## <font color=blue>Why Deep Learning over Traditional Machine Learning?

https://towardsdatascience.com/why-deep-learning-is-needed-over-traditional-machine-learning-1b6a99177063

## <font color=blue>Tips for Hyperparameter Optimization

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/#:~:text=Tips%20for%20Hyperparameter%C2%A0Optimization